In [1]:
### run 1 pilih file masterdata terupdate

import pandas as pd
import numpy as np
import os
import glob


# list_of_files = glob.glob('D:\1. Masterdata\Clean Data/*.csv')
list_of_files=os.listdir(r"D:\1. Masterdata\Clean Data")
a=[r"D:\1. Masterdata\Clean Data/"+i for i in list_of_files]
latest_file = max(a, key=os.path.getctime)
latest_file

data_all = pd.read_csv(r"D:\1. Masterdata\Clean Data\data_all_23 May With Order Online.csv", sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})
data_all['Phone'] = data_all['Phone'].astype(str).str.replace('="', '', regex = False)
data_all['Phone'] = data_all['Phone'].astype(str).str.replace('"', '', regex = False)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (12,15,20,24,25,27,29,30,31,32,33,34,35,36,37,39,53,54,57,59,60,66,67,73,74,80,81,87,88,94,95,101,102,108,122,123,134,135,136,153,159,161,167,168,169,172,173,174,175,176,180,181,183,196,197,198,199,200,201) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# run-2
j_conv = pd.read_excel("Settlement Project\Rumus Realisasi.xlsx", sheet_name = 'Paket Brand')
j_conv = j_conv.iloc[:,:5]
j_conv = j_conv.rename(columns = {'SKU' : 'Real SKU', 'SKU.1' : 'SKU Produk'})
j_conv = j_conv[j_conv['Real SKU'].notnull()]
j_conv['SKU Produk'] = j_conv['SKU Produk'].astype(str).str.replace('.0', '',regex = False)

temp_all = data_all.copy()

temp_all['Real SKU'] = temp_all['Real SKU'].astype(str)
j_conv['Real SKU'] = j_conv['Real SKU'].astype(str)

temp_all = temp_all.merge(j_conv, how = 'left', on = 'Real SKU')

paket_j = temp_all[temp_all['SKU Produk'].notnull()]

paket_j['Bundle Name'] = paket_j['Product Name']
paket_j['Product Name'] = paket_j['Name']
paket_j['SKU'] = paket_j['SKU Produk']
paket_j['Qty. Invoiced'] = paket_j['Qty. Invoiced'] * paket_j['Konversi ke UOM'].astype(float)
paket_j['Bundle Flag'] = np.nan
    
import time
import pandas as pd
import numpy as np
import math

import smtplib 
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os
    

print('A')
data_SKU = pd.read_excel(r"D:\1. Masterdata\SKU_File\data_SKU.xlsx")
print('A')
s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx", 'wb') as output:
    output.write(r.content)

if os.path.isfile(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx") :    
    SKU_append = pd.read_excel(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx")
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU.append(SKU_append, ignore_index = True, sort = False)

# data_SKU.loc[data_SKU['Price List NFI'].isin(['-']), 'Price List NFI'] = 0    
data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
if 'Real SKU_y' in paket_j.columns:
    paket_j = paket_j.drop('Real SKU_y', axis = 1)
paket_j['SKU'] = paket_j['SKU'].astype(str)
paket_j[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']] = paket_j.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(paket_j.index)[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y', 'Price List NFI_y']]

paket_j['Price List NFI']=paket_j['Price List NFI'].astype(int)
paket_j['Total Net'] = paket_j['Price List NFI'] * paket_j['Qty. Invoiced']

paket_j['Subtotal'] = paket_j['Subtotal'] * paket_j['Total Net']/paket_j.groupby(['Order #', 'Bundle Name'])['Total Net'].transform('sum')
paket_j['Selling Price'] = paket_j['Subtotal']/paket_j['Qty. Invoiced']
paket_j['Total'] = paket_j['Total'] * paket_j['Total Net']/paket_j.groupby(['Order #', 'Bundle Name'])['Total Net'].transform('sum')

paket_j['Seller Discount'] = paket_j['Seller Discount'] * paket_j['Total Net']/paket_j.groupby(['Order #', 'Bundle Name'])['Total Net'].transform('sum')
paket_j['Shipping'] = paket_j['Shipping'] * paket_j['Total Net']/paket_j.groupby(['Order #', 'Bundle Name'])['Total Net'].transform('sum')

paket_j = paket_j.drop(['Nama Paket', 'SKU Produk', 'Name', 'Konversi ke UOM'], axis = 1)
data_all = data_all.append(paket_j, ignore_index = True, sort = False)
indeks = data_all[data_all['Product Name'].isin(paket_j['Bundle Name'])].index.to_list()
data_all['Brand'][indeks] = 'Bundle'

data_all['PL Before PPN'] = data_all['Price List NFI']/1.11
data_all['Total Net Before PPN'] = data_all['PL Before PPN'] * data_all['Qty. Invoiced']

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

A
A


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [3]:
## run for check, jika ingin di unbundling isi rumus realisasi xlsx sheet paket brand
data_all[(data_all['Real SKU'].str.contains('J'))&~(data_all['Real SKU'].isin(j_conv['Real SKU'].unique()))][['Year','Real SKU','Real Nama Produk']].drop_duplicates().values


array([[2023.0, '(J)71110138', 'Box Packing NutriMart']], dtype=object)

In [4]:
# run -3
def replace(df, colname):
    df[colname] = df[colname].str.strip().str.replace(
    '  ', ' ').str.replace(
    '6 SCH', '6sch').str.replace(
    'HILO TEEN STRAWBERRY MILKSHAKEÂ\xa0 12Dx500G', 'HILO TEEN STRAWBERRY MILKSHAKE\xa0 12DX500G').str.replace(
    'HiLo Teen Strawberry Milkshake 500g', 'HILO TEEN STRAWBERRY MILKSHAKE\xa0 12DX500G').str.replace(
    'HILO TEEN STRAWBERRY MILKSHAKE\xa0 12Dx500gr', 'HILO TEEN STRAWBERRY MILKSHAKE\xa0 12DX500G').str.replace(
    'HILO TEEN STRAWBERRY MILKSHAKE\xa0 12Dx500G', 'HILO TEEN STRAWBERRY MILKSHAKE\xa0 12DX500G').str.replace(
    "W'Dank", "W'dank").str.replace(
    'L-men', 'L-Men').str.replace(
    "Hilo", "HiLo").str.replace(
    "Sch", "sch").str.replace(
    "Ml", "ml").str.replace(
    "Gr", "gr").str.replace(
    "DIABTX", "Diabtx").str.replace(
    "Empon-empon", "Empon-Empon").str.replace(
    "Nutrisari", "NutriSari").str.replace(
    "X", "x").str.replace(
    "original", "Original").str.replace(
    "hilo", "HiLo").str.replace(
    "Bbq", "BBQ").str.replace(
    "Rtd", "RTD").str.replace(
    'Turmeric 180gr', 'Turmeric (6 sch)').str.replace(
    'L-Men Gainmass Taro 225gr', 'L-Men Gain Mass Taro 225 gr').str.replace(
    'L-Men Lose Weight Chocolate Cereal (12sch)', 'L-Men Lose Weight Chocolate Cereal 300gr').str.replace(
    ' (+kaos)', ' (+Kaos)', regex = False).str.replace(
    '40 sch', '40sch', regex = False).str.replace(
    '(40 sch)', '(40sch)', regex = False).str.replace(
    'Coklat', 'Chocolate', regex = False).str.replace(
    'school', 'School', regex = False).str.replace(
    '50 sachet', '(50sch)', regex = False).str.replace(
    'tetrapack', 'Tetrapack', regex = False).str.replace(
    "10\'s$", '(10sch)', regex = True).str.replace(
    "10 sch$", '(10sch)', regex = True).str.replace(
    "(10 sch)", '(10sch)', regex = False).str.replace(
    "[", '(', regex = False).str.replace(
    "]", ')', regex = False).str.replace(
    "NutriSari Mango Smoothie Ready to Drink Jus 200ml", 'NutriSari Mango Smoothie 200ml', regex = False).str.replace(
    'L-MEN', 'L-Men',  regex = False).str.replace(
    "NutriSari Jeruk Nipis 40'sachet (4 Renceng)", 'NutriSari Jeruk Nipis (40sch)',  regex = False).str.replace(
    " - 12gr Protein / Serving", '',  regex = False)
    
    return df[colname]

In [212]:
# run 4 iter var kota
kota = 'Kaltim'

if kota == 'Jawa Timur':
    channel = 'Order Online Surabaya'
    area = 'JATIM'
    cek_cod = 'Jatim'
    vouCode='JATIM'
elif kota == 'Jawa Tengah' : ##
    channel = 'Order Online Jateng'
    area = 'JATENG'
    cek_cod = 'Jateng'
    vouCode='JATENG'
elif kota == 'Bali' :
    channel = 'Order Online Bali'
    area = 'BALI'
    cek_cod = 'Bali'
    vouCode='BALI'
elif kota == 'Kaltim' : ##
    channel = 'Order Online Samarinda'
    area = 'KALTIM (SAMARINDA)'
    cek_cod = 'Kaltim'
    vouCode='KALTIM'
elif kota == 'Lampung' :
    channel = 'Order Online Lampung'
    area = 'LAMPUNG'
    cek_cod = 'Lampung'
    vouCode='LMP'
elif kota == 'Makasar' :
    channel = 'Order Online Makassar'
    area = 'SULSEL (MAKASAR)'
    cek_cod = 'Sulsel'
    vouCode='SULSEL'
elif kota == 'Medan' : ##
    channel = 'Order Online Medan'
    area = 'SUMUT (MEDAN)'
    cek_cod = 'Sumut'
    vouCode='SUMUT'
elif kota == 'Palembang' :
    channel = 'Order Online Sumsel'
    area = 'PALEMBANG'
    cek_cod = 'Sumsel'
    vouCode='SUMSEL'
elif kota == 'Pekanbaru' : ##
    channel = 'Order Online Pekanbaru'
    area = 'Pekanbaru'
    cek_cod = 'Pekanbaru'
    vouCode='PKU'
elif kota == 'Banjarmasin' : ##
    channel = 'Order Online Banjarmasin'
    area = 'KALSEL'
    cek_cod = 'Kalsel'
    vouCode='BJMR'
elif kota == 'Jawa Barat' : ##
    channel = 'Order Online Jabar'
    area = 'JABAR'
    cek_cod = 'Jabar'
    vouCode='JABAR'
elif kota == 'Jawa Timur 2020' :
    channel = 'Order Online Surabaya'
    area = 'JATIM'
elif kota == 'Jawa Tengah 2020' :
    channel = 'Order Online Jateng'
    area = 'JATENG'

In [213]:
# run-4 change date ### cut off benna
ordol = data_all[
    (data_all['Channel'] == channel) &
    (data_all['Year'] > 2022) & 
    (data_all['True datetime'] < '2023-05-22') 
#     (data_all['True datetime'] < '2021-08-01')
#     (data_all['Month'] != 'August') &
#     (data_all['Month'] != 'September') 
]
print(channel)

Order Online Samarinda


In [214]:
# real_sku = ['1101569453P4']
# prod_name = ["NutriSari Jeruk Nipis 40'sachet (4 Renceng)",'NutriSari Jeruk Nipis (40sch)']

# temp2 = ordol[(ordol['Real SKU'].isin(real_sku)) &(ordol['Product Name'].isin(prod_name))]


# if temp2.size>0:
#     temp2 = temp2.drop('Real SKU_y', axis = 1)

#     data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')

#     data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
#     data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)    

#     list_col = ['Real SKU', 'Real Nama Produk']

#     temp2['SKU'] = '1101558453P4'

#     temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp2.index)
#     for i in list_col:
#         temp2[i] = temp_merge[str(i) + '_y']

#     list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()
#     temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(temp2.index)
#     list_pcs = [x for x in temp_merge.columns if 'PCS' in x and '_y' in x]
#     for i in list_pcs:
#         temp_merge[i] = temp_merge[i] * temp_merge['Qty. Invoiced']

#     list_col = data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()    
#     for i in list_col:
#         temp2[i] = temp_merge[str(i) + '_y']


#     print("Unbundling")
#     data_bundle1 = temp2[~temp2['Produk 1'].isnull()]
#     data_bundle1['Bundle Name'] = data_bundle1['Product Name']
#     data_bundle1['Product Name'] = data_bundle1['Produk 1']
#     data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
#     data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
#     data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
#     data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
#     data_bundle1['Bundle Flag'] = np.nan

#     data_bundle2 = temp2[~temp2['Produk 2'].isnull()]
#     data_bundle2['Bundle Name'] = data_bundle2['Product Name']
#     data_bundle2['Product Name'] = data_bundle2['Produk 2']
#     data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
#     data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
#     data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
#     data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
#     data_bundle2['Bundle Flag'] = np.nan

#     data_bundle3 = temp2[~temp2['Produk 3'].isnull()]
#     data_bundle3['Bundle Name'] = data_bundle3['Product Name']
#     data_bundle3['Product Name'] = data_bundle3['Produk 3']
#     data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
#     data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
#     data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
#     data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
#     data_bundle3['Bundle Flag'] = np.nan

#     data_bundle4 = temp2[~temp2['Produk 4'].isnull()]
#     data_bundle4['Bundle Name'] = data_bundle4['Product Name']
#     data_bundle4['Product Name'] = data_bundle4['Produk 4']
#     data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
#     data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
#     data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
#     data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
#     data_bundle4['Bundle Flag'] = np.nan

#     data_bundle5 = temp2[~temp2['Produk 5'].isnull()]
#     data_bundle5['Bundle Name'] = data_bundle5['Product Name']
#     data_bundle5['Product Name'] = data_bundle5['Produk 5']
#     data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
#     data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
#     data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
#     data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
#     data_bundle5['Bundle Flag'] = np.nan

#     data_bundle6 = temp2[~temp2['Produk 6'].isnull()]
#     data_bundle6['Bundle Name'] = data_bundle6['Product Name']
#     data_bundle6['Product Name'] = data_bundle6['Produk 6']
#     data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
#     data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
#     data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
#     data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
#     data_bundle6['Bundle Flag'] = np.nan

#     data_bundle7 = temp2[~temp2['Produk 7'].isnull()]
#     data_bundle7['Bundle Name'] = data_bundle7['Product Name']
#     data_bundle7['Product Name'] = data_bundle7['Produk 7']
#     data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
#     data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
#     data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
#     data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
#     data_bundle7['Bundle Flag'] = np.nan

#     data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
#     data_bundle['SKU'] = data_bundle['SKU'].astype(str)
#     data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
#     data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

#     print("Pricing")
#     temp2 = temp2.append(data_bundle, ignore_index = True, sort = False)
#     list_col = ['Real SKU', 'Price List NFI', 'Harga Cost']

#     temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp2.index)
#     for i in list_col:
#         temp2[i] = temp_merge[str(i) + '_y']

#     temp2['Price List NFI'] = pd.to_numeric(temp2['Price List NFI']).astype(int)
#     temp2['Harga Cost'] = pd.to_numeric(temp2['Harga Cost']).astype(int)
#     temp2['Qty. Invoiced'] = pd.to_numeric(temp2['Qty. Invoiced']).astype(int)

#     temp2['Total Net'] = temp2['Price List NFI'] * temp2['Qty. Invoiced']
#     temp2['Total Harga Cost'] = temp2['Harga Cost'] * temp2['Qty. Invoiced']

#     list_bundle = temp2[temp2['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
#     list_nobundle = temp2[temp2['Bundle Name'].notnull()]
#     list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
#     list_nobundle

#     temp2['Total'][list_nobundle.index] = list_nobundle['Total_y']
#     temp2['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

#     temp = temp2[temp2['Bundle Name'].notnull()]
#     temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
#     temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
#     temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

#     temp2['Total'][temp.index] = temp['Total']
#     temp2['Subtotal'][temp.index] = temp['Subtotal']

#     temp2['Order #'] = temp2['Order #'].astype(str).str.replace('.0', '', regex = False)

#     list_bundle = temp2[temp2['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
#     list_nobundle = temp2[temp2['Bundle Name'].notnull()]
#     list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
#     list_nobundle

#     temp2['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
#     temp = temp2[temp2['Bundle Name'].notnull()]
#     temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
#     temp2['Seller Discount'][temp.index] = temp['Seller Discount']


#     temp = temp2[temp2['Bundle Name'].isnull()]
#     temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

#     temp = temp2.merge(temp_group, how = 'left', on = 'Order #').set_index(temp2.index)
#     temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

#     temp2['Shipping'][temp.index] = temp['Shipping_y']
#     list_bundle = temp2[temp2['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
#     list_nobundle = temp2[temp2['Bundle Name'].notnull()]
#     list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
#     list_nobundle

#     temp2['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
#     temp = temp2[temp2['Bundle Name'].notnull()]
#     temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
#     temp2['Shipping'][temp.index] = temp['Shipping']

#     indeks = ordol[(ordol['Real SKU'].isin(real_sku)) &
#         (ordol['Product Name'].isin(prod_name)) &
#                       (ordol['Order #'].isin(temp2['Order #'].unique()))].index.to_list()

#     prod_name = ordol[(ordol['Real SKU'].isin(real_sku)) & 
#                       (ordol['Order #'].isin(temp2['Order #'].unique()))]['Product Name'].unique()

#     ordol = ordol.drop(indeks, axis = 0)

#     indeks = ordol[(ordol['Bundle Name'].isin(prod_name)) & 
#                       (ordol['Order #'].isin(temp2['Order #'].unique()))].index.to_list()

#     ordol = ordol.drop(indeks, axis = 0)
#     # ordol = ordol.reset_index(drop = True)
#     # orders = temp2['Order #'].unique()

#     # ordol = ordol.drop(indeks, axis = 0)
#     ordol = ordol.append(temp2, ignore_index = True, sort = False)

In [215]:
# temp2[['Order #', 'Product Name','Bundle Name','Real SKU', 'Real Nama Produk', 'Qty. Invoiced']]

# Repairing Product Price and Discount

In [216]:
## run -5
selcen = pd.DataFrame()

for i in os.listdir('Settlement Project/Order Online/' + kota + '/Seller Centre'):
    print('Settlement Project/Order Online/' + kota + '/Seller Centre/' + i)
    temp_read = pd.read_csv('Settlement Project/Order Online/' + kota + '/Seller Centre/' + i)
    temp_read['order_id'] = temp_read['order_id'].fillna(method='ffill')
    temp_read = temp_read.drop('quantity', axis = 1)
    temp = temp_read.drop_duplicates('order_id').drop('product', axis = 1)
    temp_read = temp_read[['order_id', 'product']]
    temp_read = temp_read.merge(temp, how = 'left', on = 'order_id')
    if len(selcen) != 0:
        temp_read = temp_read[~temp_read['order_id'].isin(selcen['order_id'])]
    selcen = selcen.append(temp_read, ignore_index = True, sort = False)
    
selcen_ori = selcen.copy()

Settlement Project/Order Online/Kaltim/Seller Centre/orderonline_orders_all_products_01-01-2021_31-08-2021.csv
Settlement Project/Order Online/Kaltim/Seller Centre/orderonline_orders_all_products_01-26 Apr.csv
Settlement Project/Order Online/Kaltim/Seller Centre/orderonline_orders_all_products_01-28 Feb.csv
Settlement Project/Order Online/Kaltim/Seller Centre/orderonline_orders_all_products_01-30 Nov2021.csv
Settlement Project/Order Online/Kaltim/Seller Centre/orderonline_orders_all_products_01-30 Sept 2021.csv
Settlement Project/Order Online/Kaltim/Seller Centre/orderonline_orders_all_products_01-31 Desember 2021.csv
Settlement Project/Order Online/Kaltim/Seller Centre/orderonline_orders_all_products_01-31 Jan 2022.csv
Settlement Project/Order Online/Kaltim/Seller Centre/orderonline_orders_all_products_01-31 Mar.csv
Settlement Project/Order Online/Kaltim/Seller Centre/orderonline_orders_all_products_01-31 Okt 2021.csv
Settlement Project/Order Online/Kaltim/Seller Centre/orderonline_or

In [217]:
## run 6

# selcen['order_id'] = selcen['order_id'].fillna(method='ffill')
# selcen = selcen.drop('quantity', axis = 1)
# temp = selcen.drop_duplicates('order_id').drop('product', axis = 1)
# selcen = selcen[['order_id', 'product']]
# selcen = selcen.merge(temp, how = 'left', on = 'order_id')

selcen['order_id'] = selcen['order_id'].astype(int)
selcen['product'] = selcen['product'].astype(str).str.replace('L-Men Hi Protein 2 Go Chocolate (24 pcs) - 12gr Protein / Serving', 'L-Men Hi Protein 2 Go Chocolate (24 pcs)', regex = False)
selcen['product'] = selcen['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
selcen['product'] = selcen['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
selcen['product'] = selcen['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)

product = selcen['product'].str.split("\(x",1, expand = True)
indeks = product[~product[1].astype(str).str.contains(')', regex = False)].index.to_list()
product[1][indeks] = '1)'
selcen['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
selcen['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch')

In [218]:
## run 7 must True
len(selcen_ori) == len(selcen)
# selcen['Quantity'].unique()

True

In [219]:
## run -8
indeks = selcen_ori[~selcen_ori['product'].astype(str).str.contains('(x', regex = False)].index.to_list()

list_alias_name = [x for x in data_SKU.columns if 'Alias Nama' in x]

for i in indeks:
    for j in list_alias_name:
        if str(selcen['product'][i]).lower() in data_SKU[j].astype(str).str.lower().values:
            print(selcen['product'][i])
            selcen['product'][i] = data_SKU['Nama Produk'].loc[str(selcen['product'][i]).lower() == data_SKU[j].astype(str).str.lower()].values[0]


In [220]:
## run -9
selcen_short = selcen[['order_id', 'product', 'status','product_price', 'discount', 'shipping_cost', 'Quantity', 'created_at']]
selcen_short['product'] = replace(selcen_short, 'product')
ordol['Product Name'] = replace(ordol, 'Product Name')
ordol['Bundle Name'] = replace(ordol, 'Bundle Name')

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCop

In [221]:
#notes
# ordol dari mastedata
# j_conv = konverter paket j
# selcen_short = data selcen simple
# paket_j =  data selcen yang beli pake j + unbundling
# selcen short diappend paket j -> data selcen di unbundling
# ordol merge selcen x masterdata

In [222]:
# run 10
j_conv = pd.read_excel("Settlement Project\Rumus Realisasi.xlsx", sheet_name='Paket Brand')
j_conv = j_conv.iloc[:, :5]
j_conv = j_conv.rename(columns={'SKU': 'Real SKU', 'SKU.1': 'SKU Produk'})
j_conv = j_conv[j_conv['Real SKU'].notnull()]
j_conv['SKU Produk'] = j_conv['SKU Produk'].astype(str).str.replace('.0', '', regex=False)

selcen_short['order_id'] = selcen_short['order_id'].astype(str)

selcen_short = selcen_short.rename(columns={
                                   'order_id': 'Order #', 'product': 'Product Name', 'status': 'Status Seller Centre'})

j_conv = j_conv.merge(ordol[ordol['Bundle Name'].isnull()].groupby(['Product Name'])['Real SKU'].first().reset_index(), how='left', on='Real SKU')

temp_all = selcen_short.merge(j_conv, how='left', on='Product Name')

paket_j = temp_all[temp_all['SKU Produk'].notnull()]

paket_j['Product Name'] = paket_j['Name']
paket_j['SKU'] = paket_j['SKU Produk']
paket_j['Quantity'] = paket_j['Quantity'] * \
    paket_j['Konversi ke UOM'].astype(float)

data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
if 'Real SKU_y' in paket_j.columns:
    paket_j = paket_j.drop('Real SKU_y', axis=1)
paket_j['SKU'] = paket_j['SKU'].astype(str)

paket_j = paket_j.drop(['Nama Paket', 'SKU Produk', 'Name', 'Konversi ke UOM'], axis=1)

paket_j['product_price'] = paket_j['product_price'].fillna(0) * paket_j['Quantity']/paket_j.groupby(['Order #'])['Quantity'].transform('sum')
paket_j['discount'] = paket_j['discount'].fillna(0) * paket_j['Quantity']/paket_j.groupby(['Order #'])['Quantity'].transform('sum')
paket_j['shipping_cost'] = paket_j['shipping_cost'].fillna(0) * paket_j['Quantity']/paket_j.groupby(['Order #'])['Quantity'].transform('sum')

selcen_short = selcen_short.append(paket_j, ignore_index=True, sort=False)
selcen_short = selcen_short.drop(['Real SKU', 'SKU'], axis=1)



ordol_real = ordol[ordol['Bundle Name'].isnull()]
ordol_bundle = ordol[ordol['Bundle Name'].notnull()]

ordol_real['Order #'] = ordol_real['Order #'].astype(str)

ordol_merge = ordol_real.merge(selcen_short, how='left', on=[
                               'Order #', 'Product Name'])
ordol_merge['Real Price'] = ordol_merge['product_price'].fillna(0) * ordol_merge['Total Net Before PPN']/ordol_merge.groupby(['Order #'])['Total Net Before PPN'].transform('sum')
ordol_merge['Real Discount'] = ordol_merge['discount'].fillna(0) * ordol_merge['Total Net Before PPN']/ordol_merge.groupby(['Order #'])['Total Net Before PPN'].transform('sum')
ordol_merge['Real Shipping'] = ordol_merge['shipping_cost'].fillna(0) * ordol_merge['Total Net Before PPN']/ordol_merge.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

bundle_merge = ordol_bundle.merge(ordol_merge[['Order #', 'created_at', 'Product Name', 'Status Seller Centre', 'Real Price', 'Real Discount', 'Real Shipping']].drop_duplicates(['Order #', 'Product Name']), how='left',
                                  left_on=['Order #', 'Bundle Name'], right_on=['Order #', 'Product Name'])
bundle_merge = bundle_merge.rename(columns={'Product Name_x': 'Product Name'})
bundle_merge = bundle_merge.drop('Product Name_y', axis=1)

bundle_merge['Real Price'] = bundle_merge['Real Price'].fillna(0) * bundle_merge['Total Net Before PPN']/bundle_merge.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundle_merge['Real Discount'] = bundle_merge['Real Discount'].fillna(0) * bundle_merge['Total Net Before PPN']/bundle_merge.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundle_merge['Real Shipping'] = bundle_merge['Real Shipping'].fillna(0) * bundle_merge['Total Net Before PPN']/bundle_merge.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')

ordol_merge = ordol_merge.append(bundle_merge, ignore_index=True, sort=False)


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Warehouse

In [223]:
## run 11
wms = selcen_short.merge(ordol_merge[ordol_merge['Bundle Name'].isnull()].groupby(['Order #', 'Product Name'])['Real SKU'].first().reset_index(), how = 'left', on = ['Order #', 'Product Name'])

list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()
wms.loc[wms['Product Name']=="BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng",'Real SKU']='PN77B57'
wms = wms.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
list_pcs = [x for x in wms.columns if 'PCS' in x]
for i in list_pcs:
    wms[i] = wms[i] * wms['Quantity']

print("Unbundling")
data_bundle1 = wms[~wms['Produk 1'].isnull()]
data_bundle1['Real SKU'] = data_bundle1['SKU Produk 1']
data_bundle1['Quantity'] = data_bundle1['PCS Produk 1']

data_bundle2 = wms[~wms['Produk 2'].isnull()]
data_bundle2['Real SKU'] = data_bundle2['SKU Produk 2']
data_bundle2['Quantity'] = data_bundle2['PCS Produk 2']

data_bundle3 = wms[~wms['Produk 3'].isnull()]
data_bundle3['Real SKU'] = data_bundle3['SKU Produk 3']
data_bundle3['Quantity'] = data_bundle3['PCS Produk 3']

data_bundle4 = wms[~wms['Produk 4'].isnull()]
data_bundle4['Real SKU'] = data_bundle4['SKU Produk 4']
data_bundle4['Quantity'] = data_bundle4['PCS Produk 4']

data_bundle5 = wms[~wms['Produk 5'].isnull()]
data_bundle5['Real SKU'] = data_bundle5['SKU Produk 5']
data_bundle5['Quantity'] = data_bundle5['PCS Produk 5']

data_bundle6 = wms[~wms['Produk 6'].isnull()]
data_bundle6['Real SKU'] = data_bundle6['SKU Produk 6']
data_bundle6['Quantity'] = data_bundle6['PCS Produk 6']

data_bundle7 = wms[~wms['Produk 7'].isnull()]
data_bundle7['Real SKU'] = data_bundle7['SKU Produk 7']
data_bundle7['Quantity'] = data_bundle7['PCS Produk 7']

data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)

# wms_j = paket_j.merge(ordol_merge.groupby(['Order #', 'Product Name'])['Real SKU'].first().reset_index(), how = 'left', on = ['Order #', 'Product Name'])
# data_bundle = data_bundle.append(wms_j, ignore_index = True, sort = False)

wms_j = paket_j.drop('Real SKU', axis = 1).rename(columns = {'SKU' : 'Real SKU'})
data_bundle = data_bundle.append(wms_j, ignore_index = True, sort = False)

Unbundling


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [224]:
## run 12
ordol_bundle = ordol_merge[ordol_merge['Bundle Name'].notnull()]

### timo mar 23
tempord=data_bundle[data_bundle['Real SKU']=='2HF1401250']['Order #'].unique()
ordol_bundle.loc[(ordol_bundle['Real SKU']=='21014281')&
             (ordol_bundle['Order #'].isin(tempord)),'Real SKU']='2HF1401250'#[['Order #', 'Real SKU','Real Nama Produk']]
### timo mar 23 

ordol_bundle = ordol_bundle.merge(data_bundle[['Order #', 'Real SKU', 'Quantity']].drop_duplicates(['Order #', 'Real SKU']), how = 'left', on = ['Order #', 'Real SKU']).set_index(ordol_bundle.index)
ordol_merge['Quantity'][ordol_bundle.index] = ordol_bundle['Quantity_y']

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [225]:
### run -13 must null

ordol_merge[(ordol_merge['Quantity'].isnull())&(ordol_merge['Order #'].isin(selcen_short['Order #'].unique()))][['Order #','Product Name','SKU','Real SKU']].drop_duplicates()#['Real Nama Produk'].unique()#.columns
## ini harus null ya


,Order #,Product Name,SKU,Real SKU


# Cek Voucher

In [226]:
## run -14 blh di run sekali aja
voucher = pd.read_excel(r"Settlement Project\Order Online\Settement transfer, Kode Voucher\Master Kode Voucher (NHD organik dan orderonline).xlsx")

In [227]:
voucher['Kategori Voucher'].unique()

array(['Shipping ', 'Pengembalian', 'Product', 'Shipping', 'Product ',
       'ga pernah dipake trial doang', 'Product + shipping (manual)', nan,
       'diskon produk'], dtype=object)

In [228]:
## run -14.1
vouShip23 = pd.read_excel(r"Settlement Project\Order Online\Settement transfer, Kode Voucher\Master Kode Voucher (NHD organik dan orderonline).xlsx",sheet_name='2023')

In [229]:
## run -15
ordol_merge['Discount Voucher Product'] = np.nan

# voucher_product = voucher[voucher['Kategori Voucher'] == 'Product']['coupon'].unique()
# indeks = ordol_merge[ordol_merge['Coupon Code'].isin(voucher_product)].index.to_list()
# ordol_merge['Discount Voucher Product'][indeks] = ordol_merge['Real Discount'][indeks]
# ordol_merge['Discount Voucher Product'] = ordol_merge['Discount Voucher Product'].fillna(0)

ordol_merge['Discount Voucher Shipping'] = np.nan
voucher_shipping = voucher[voucher['Kategori Voucher'].isin(['Shipping','Shipping '])]['coupon'].unique()
indeks = ordol_merge[ordol_merge['Coupon Code'].isin(voucher_shipping)].index.to_list()
ordol_merge['Discount Voucher Shipping'][indeks] = ordol_merge['Real Discount'][indeks]

## timo apr 2023 -> add other loop & change year if there is new program
for vou in vouShip23['Shipping Code Indicator'].unique():
    indeks = ordol_merge[(ordol_merge['Year']==2023)&(ordol_merge['Coupon Code'].str.contains(vou,case=False,na=False))].index.to_list()
    ordol_merge['Discount Voucher Shipping'][indeks] = ordol_merge['Real Discount'][indeks]
    
    
ordol_merge['Discount Voucher Shipping'] = ordol_merge['Discount Voucher Shipping'].fillna(0)
ordol_merge_copy=ordol_merge.copy()

indeks = ordol_merge[ordol_merge['Coupon Code'].notnull()].index.to_list()
ordol_merge['Discount Voucher Product'][indeks] = ordol_merge['Real Discount'][indeks] - ordol_merge['Discount Voucher Shipping'][indeks]
ordol_merge['Discount Voucher Product'] = ordol_merge['Discount Voucher Product'].fillna(0)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:

In [230]:
## run -16
voucher['Coupon Code'] = voucher['coupon']
voucher['Cluster'] = np.nan ##26 sep
ordol_merge = ordol_merge.merge(voucher[['Coupon Code', 'Pemilik Budget', 'Cluster']].drop_duplicates('Coupon Code'), how = 'left', on = 'Coupon Code')
# ordol_merge = ordol_merge.merge(voucher[['Coupon Code', 'Pemilik Budget']].drop_duplicates('Coupon Code'), how = 'left', on = 'Coupon Code')

In [231]:
## run -17
arearun=area
temp = ordol_merge[(ordol_merge['Year'] > 2021) & (ordol_merge['Pemilik Budget'].isnull()) & (ordol_merge['Coupon Code'].notnull())]

area = pd.read_excel(r"Settlement Project\Nutrimart\voucher.xlsx", sheet_name = 'Area')
cluster = pd.read_excel(r"Settlement Project\Nutrimart\voucher.xlsx", sheet_name = 'Cluster')

temp_coupon = temp[['Coupon Code']].drop_duplicates()

temp_coupon['Coupon Code'] = temp_coupon['Coupon Code'].astype(str).str.upper()
temp_coupon['Dept'] = np.nan
temp_coupon['Cluster Code'] = np.nan

temp_coupon['Original Coupon Code'] = temp_coupon['Coupon Code'].copy()

for i in area['Kode Area']:
    indeks = temp_coupon[temp_coupon['Coupon Code'].astype(str).str.upper().str.contains('^' + str(i), regex = True)].index.to_list()
    if len(indeks) > 0 :
        temp_coupon['Dept'][indeks] = 'Homdel'
        temp_coupon['Coupon Code'][indeks] = temp_coupon['Coupon Code'][indeks].astype(str).str.upper().str.replace('^' + str(i), '', regex = True)
        temp_coupon['Cluster Code'][indeks] = temp_coupon['Coupon Code'][indeks].astype(str).str[0]
        indeks_2 = temp_coupon[~(temp_coupon['Cluster Code'].astype(str).str.isnumeric()) & (temp_coupon.index.isin(indeks))].index.to_list()
        temp_coupon['Cluster Code'][indeks_2] = temp_coupon['Coupon Code'][indeks_2].astype(str).str[:3]
        
temp_coupon_null = temp_coupon[temp_coupon['Dept'].isnull()].copy()
temp_coupon_null['Dept'] = temp_coupon_null['Coupon Code'].astype(str).str[:3]
temp_coupon_null['Cluster Code'] = temp_coupon_null['Coupon Code'].astype(str).str[3:4]

indeks_3 = temp_coupon_null[~(temp_coupon_null['Cluster Code'].astype(str).str.isnumeric())].index.to_list()
temp_coupon_null['Cluster Code'][indeks_3] = temp_coupon_null['Coupon Code'][indeks_3].astype(str).str[3:6]

temp_coupon['Dept'][temp_coupon_null.index] = temp_coupon_null['Dept']
temp_coupon['Cluster Code'][temp_coupon_null.index] = temp_coupon_null['Cluster Code']

cluster['Cluster Code'] = cluster['Cluster Code'].astype(str)
temp_coupon['Cluster Code'] = temp_coupon['Cluster Code'].astype(str)

temp_coupon = temp_coupon.merge(cluster, how = 'left', on = 'Cluster Code')

temp = temp.drop(['Pemilik Budget', 'Cluster'], axis = 1)
temp_coupon['Pemilik Budget'] = temp_coupon['Cluster']
temp_coupon['Coupon Code'] = temp_coupon['Original Coupon Code']

## timo jan 23
deptlist=pd.read_excel(r"Settlement Project\Order Online\Settement transfer, Kode Voucher\deptlist_nfi.xlsx")
temp_coupon.loc[temp_coupon['Dept'].isin(deptlist['deptlist'].unique()),'Pemilik Budget']=temp_coupon['Dept']
temp_coupon.loc[~temp_coupon['Dept'].isin(deptlist['deptlist'].unique()),'Pemilik Budget']="ECOM"

temp = temp.merge(temp_coupon[['Coupon Code', 'Pemilik Budget', 'Cluster']].drop_duplicates('Coupon Code'), how = 'left', on = 'Coupon Code').set_index(temp.index)

ordol_merge['Pemilik Budget'][temp.index] = temp['Pemilik Budget']
ordol_merge['Cluster'][temp.index] = temp['Cluster']
area=arearun

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

In [232]:
### run 17.1
ordol_merge.loc[(ordol_merge['Coupon Code'].str.contains(vouCode,case=False,na=False))&(ordol_merge['Pemilik Budget']=='MEA'),'Pemilik Budget']='Homdel'
ordol_merge.loc[(ordol_merge['Pemilik Budget']=='homdel'),'Pemilik Budget']='Homdel'


In [233]:
vouCode
temp_coupon#[temp_coupon['Order #']=='105211010']

,Coupon Code,Dept,Cluster Code,Original Coupon Code,Cluster,Pemilik Budget
0,SMDRSLB008,Homdel,RSL,SMDRSLB008,Reseller,Homdel
1,YDLSMDY001,YDL,SMD,YDLSMDY001,NaN,YDL
2,YDLSMDY006,YDL,SMD,YDLSMDY006,NaN,YDL
3,SMDRSLB010,Homdel,RSL,SMDRSLB010,Reseller,Homdel
4,YDLSMDY007,YDL,SMD,YDLSMDY007,NaN,YDL
5,SMDRSLB009,Homdel,RSL,SMDRSLB009,Reseller,Homdel
6,QBTBD23KALTIM287,QBT,BD2,QBTBD23KALTIM287,NaN,QBT
7,SMDRSLB002,Homdel,RSL,SMDRSLB002,Reseller,Homdel
8,SMDRSLA036,Homdel,RSL,SMDRSLA036,Reseller,Homdel
9,QBHDOHKAL12,QBH,DOH,QBHDOHKAL12,NaN,QBH


# Payment Transfer

In [234]:
area

'KALTIM (SAMARINDA)'

In [235]:
## run -18
if '2020' in kota:
    transfer = pd.read_excel(r'Settlement Project\Order Online/Settement transfer, Kode Voucher/Mutasi Rekening (3).xlsx')
    print('2020')
    transfer = transfer[transfer['Area'] == area]
    print(area)
    transfer = transfer[transfer['Jumlah'].notnull()]
    transfer['Transfer'] = pd.to_numeric(transfer['Jumlah'].astype(str).str.replace('.00 CR','', regex = False).str.replace('.00','', regex = False).str.replace(',','', regex = False).str.replace(' ','', regex = False), errors = 'coerce').fillna(0).astype(int)
else :
    transfer1 = pd.read_excel(r'Settlement Project\Order Online/Settement transfer, Kode Voucher/Mutasi Rekening Order Online.xlsx')
    transfer2 = pd.read_excel(r'Settlement Project\Order Online/Settement transfer, Kode Voucher/Order Online-Transfer.xlsx', sheet_name = '2022')
    transfer3 = pd.read_excel(r"Settlement Project\Order Online\Settement transfer, Kode Voucher\Mutasi Rekening (18).xlsx", sheet_name = '2023')
    # print(len(transfer3))
    
    ### gabung transfer3 ke transfer 2
    if 'orde' in transfer3.columns:
        transfer3=transfer3.rename(columns={'orde':'TGL'})
    transfer3=transfer3[~transfer3['ID Order'].isin(transfer2['ID Order'].unique())][[x for x in transfer2.columns if x in transfer3.columns]]
    # print(len(transfer3))
    transfer2=transfer2.append(transfer3)
    
    transfer2 = transfer2[~transfer2['ID Order'].isin(transfer1['ID Order'])]
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('KALTIM',case=False)),'Area']='KALTIM (SAMARINDA)'
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('JATENG',case=False)),'Area']='JATENG'
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('JATIM',case=False)),'Area']='JATIM'
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('BALI',case=False)),'Area']='BALI'
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('SULSEL',case=False)),'Area']='SULSEL (MAKASAR)'
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('SUMUT',case=False)),'Area']='SUMUT (MEDAN)'
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('PEKANB',case=False)),'Area']='Pekanbaru'
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('KALSEL',case=False)),'Area']='KALSEL'
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('LAMPUN',case=False)),'Area']='Lampung'
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('SUMSEL',case=False)),'Area']='PALEMBANG'
    transfer2.loc[(transfer2['Area'].isnull())&(transfer2['Mutasi'].str.contains('JABAR',case=False)),'Area']='JABAR'
    print('')
    print('---this must null----')
    display(transfer2[(transfer2['Area'].isnull())]['Mutasi'].unique())
    print('---this must null----')
    print('')
    transfer2=transfer2.rename(columns={'TGL':'tgl','Jumlah':617}) ##### wkwwkwk liat dulu aje
    transfer = transfer1.append(transfer2, ignore_index = True, sort = False)
    transfer.loc[transfer['Area'].str.contains('Lampung',case=False,na=False),'Area']='LAMPUNG'
    transfer.loc[transfer['Area'].str.contains('KAlSEL',case=False,na=False),'Area']='KALSEL'
    print('2021')
    transfer = transfer[transfer['Area'] == area]
    print(area)
    transfer = transfer[transfer[617].notnull()]
    transfer['Transfer'] = pd.to_numeric(transfer[617].astype(str).str.replace('.00 CR','', regex = False).str.replace('.00','', regex = False).str.replace(',','', regex = False).str.replace(' ','', regex = False)).astype(int)


---this must null----


array([nan,
       'TRSF E-BANKING CR 3103/FTSCY/WS95031 239877.00 dapoer adinda FITRIA DANASARI'],
      dtype=object)

---this must null----

2021
KALTIM (SAMARINDA)


In [236]:
## run -19
transfer = transfer.rename(columns = {'ID Order' : 'Order ID'})
transfer['Order ID'] = transfer['Order ID'].astype(str).str.replace('.0', '', regex = False)

order_transfer = ordol_merge[ordol_merge['Order #'].isin(transfer['Order ID'])]
non_transfer = ordol_merge[~ordol_merge['Order #'].isin(transfer['Order ID'])]

order_transfer['Payment Fee'] = np.nan

ordol_real = order_transfer[order_transfer['Bundle Name'].isnull()]
ordol_bundle = order_transfer[order_transfer['Bundle Name'].notnull()]

ordol_real['Order #'] = ordol_real['Order #'].astype(str)
transfer['Order ID'] = transfer['Order ID'].astype(str)
transfer = transfer.rename(columns =  {'Order ID' : 'Order #'})
transfer_short = transfer[['Order #', 'Transfer']].drop_duplicates('Order #')

order_transfer = ordol_real.merge(transfer_short, how = 'left', on = ['Order #'])
order_transfer['Transfer'] = order_transfer['Transfer'].fillna(0) * order_transfer['Total Net Before PPN']/order_transfer.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

bundle_merge = ordol_bundle.merge(order_transfer[['Order #', 'Product Name','Transfer']].drop_duplicates(['Order #', 'Product Name']), how = 'left', 
                                 left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])
bundle_merge = bundle_merge.rename(columns = {'Product Name_x' : 'Product Name'})
bundle_merge = bundle_merge.drop('Product Name_y', axis = 1)

bundle_merge['Transfer'] = bundle_merge['Transfer'].fillna(0) * bundle_merge['Total Net Before PPN']/bundle_merge.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')

order_transfer = order_transfer.append(bundle_merge, ignore_index = True, sort = False)

# order_transfer['Shipping Fee'] = order_transfer['Transfer'] - (order_transfer['Real Price'] + order_transfer['Real Shipping'])
order_transfer['Shipping Fee'] = order_transfer['Discount Voucher Shipping']

order_aft_transfer = order_transfer.append(non_transfer, ignore_index = True, sort = False)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [237]:
# order_aft_transfer[order_aft_transfer['Order #'].isin(transfer['Order #'])][['Order #', 'Real Nama Produk', 'Bundle Name','Transfer']]

# COD

In [238]:
### run -20
cod = pd.DataFrame()

for i in os.listdir('Settlement Project\Order Online/' + kota + '/Settlement/'):
    if 'COD' in i or 'cod' in i:
        print('Settlement Project\Order Online/' + kota + '/Settlement/' + i)
        cod = cod.append(pd.read_csv('Settlement Project\Order Online/' + kota + '/Settlement/' + i, header = 2), ignore_index = True, sort = False)

Settlement Project\Order Online/Kaltim/Settlement/1.Export COD Revenue + Order Data Januari.csv
Settlement Project\Order Online/Kaltim/Settlement/2.Export COD Revenue + Order Data Februari.csv
Settlement Project\Order Online/Kaltim/Settlement/3.Export COD Revenue + Order Data Maret.csv
Settlement Project\Order Online/Kaltim/Settlement/4.Export COD Revenue + Order Data April.csv
Settlement Project\Order Online/Kaltim/Settlement/5.Export COD Revenue + Order Data Mei.csv
Settlement Project\Order Online/Kaltim/Settlement/Export COD Revenue + Order Data 1 Jan - 31 Aug.csv
Settlement Project\Order Online/Kaltim/Settlement/Export COD Revenue + Order Data 1-25 Apr.csv
Settlement Project\Order Online/Kaltim/Settlement/Export COD Revenue + Order Data 1-28 Feb.csv
Settlement Project\Order Online/Kaltim/Settlement/Export COD Revenue + Order Data 1-30 Nov.csv
Settlement Project\Order Online/Kaltim/Settlement/Export COD Revenue + Order Data 1-30 Sept.csv
Settlement Project\Order Online/Kaltim/Settle

In [239]:
## run -21
cod['Order ID'] = cod['Order ID'].astype(str)

order_cod = order_aft_transfer[order_aft_transfer['Order #'].isin(cod['Order ID'])]
non_cod = order_aft_transfer[~order_aft_transfer['Order #'].isin(cod['Order ID'])]

ordol_real = order_cod[order_cod['Bundle Name'].isnull()]
ordol_bundle = order_cod[order_cod['Bundle Name'].notnull()]

ordol_real['Order #'] = ordol_real['Order #'].astype(str)
cod['Order ID'] = cod['Order ID'].astype(str)
cod = cod.rename(columns =  {'Order ID' : 'Order #'})
cod_short = cod[['Order #', 'COD Fee (Rp)', 'COD Fee Tax (Rp)']].drop_duplicates('Order #')

order_cod = ordol_real.merge(cod_short, how = 'left', on = ['Order #'])
order_cod['COD Fee (Rp)'] = order_cod['COD Fee (Rp)'].fillna(0).astype(int) * order_cod['Total Net Before PPN']/order_cod.groupby(['Order #'])['Total Net Before PPN'].transform('sum')
order_cod['COD Fee Tax (Rp)'] = order_cod['COD Fee Tax (Rp)'].fillna(0).astype(int) * order_cod['Total Net Before PPN']/order_cod.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

bundle_merge = ordol_bundle.merge(order_cod[['Order #', 'Product Name','COD Fee (Rp)', 'COD Fee Tax (Rp)']].drop_duplicates(['Order #', 'Product Name']), how = 'left', 
                                 left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])
bundle_merge = bundle_merge.rename(columns = {'Product Name_x' : 'Product Name'})
bundle_merge = bundle_merge.drop('Product Name_y', axis = 1)

bundle_merge['COD Fee (Rp)'] = bundle_merge['COD Fee (Rp)'].fillna(0) * bundle_merge['Total Net Before PPN']/bundle_merge.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundle_merge['COD Fee Tax (Rp)'] = bundle_merge['COD Fee Tax (Rp)'].fillna(0) * bundle_merge['Total Net Before PPN']/bundle_merge.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')

order_cod = order_cod.append(bundle_merge, ignore_index = True, sort = False)

order_cod['Payment Fee'] = order_cod['COD Fee (Rp)'] + order_cod['COD Fee Tax (Rp)']

order_cod['Shipping Fee'] = order_cod['Discount Voucher Shipping']

order_aft_cod = order_cod.append(non_cod, ignore_index = True, sort = False)

# E Payment

In [240]:
## run -22
epayment = pd.DataFrame()

for i in os.listdir('Settlement Project\Order Online/' + kota + '/Settlement/'):
    if 'E-Payments' in i or 'payment' in i:
        print('Settlement Project\Order Online/' + kota + '/Settlement/' + i)
        epayment = epayment.append(pd.read_csv('Settlement Project\Order Online/' + kota + '/Settlement/' + i, header = 2), ignore_index = True, sort = False)

Settlement Project\Order Online/Kaltim/Settlement/1.Export E-Payments Revenue Januari.csv
Settlement Project\Order Online/Kaltim/Settlement/2.Export E-Payments Revenue Februari.csv
Settlement Project\Order Online/Kaltim/Settlement/3.Export E-Payments Revenue Maret.csv
Settlement Project\Order Online/Kaltim/Settlement/4.Export E-Payments Revenue April.csv
Settlement Project\Order Online/Kaltim/Settlement/5.Export E-Payments Revenue Mei.csv
Settlement Project\Order Online/Kaltim/Settlement/Export E-Payments Revenue 1 Jan-31 Aug.csv
Settlement Project\Order Online/Kaltim/Settlement/Export E-Payments Revenue 1-25 Apr.csv
Settlement Project\Order Online/Kaltim/Settlement/Export E-Payments Revenue 1-28 Feb.csv
Settlement Project\Order Online/Kaltim/Settlement/Export E-Payments Revenue 1-30 Nov.csv
Settlement Project\Order Online/Kaltim/Settlement/Export E-Payments Revenue 1-30 Sept.csv
Settlement Project\Order Online/Kaltim/Settlement/Export E-Payments Revenue 1-31 Des.csv
Settlement Project

In [241]:
## run -23
epayment['Order ID'] = epayment['Order ID'].astype(str)

order_epayment = order_aft_cod[order_aft_cod['Order #'].isin(epayment['Order ID'])]
non_epayment = order_aft_cod[~order_aft_cod['Order #'].isin(epayment['Order ID'])]

ordol_real = order_epayment[order_epayment['Bundle Name'].isnull()]
ordol_bundle = order_epayment[order_epayment['Bundle Name'].notnull()]

ordol_real['Order #'] = ordol_real['Order #'].astype(str)
epayment['Order ID'] = epayment['Order ID'].astype(str)
epayment = epayment.rename(columns =  {'Order ID' : 'Order #'})
epayment_short = epayment[['Order #', 'Transaction Fee', 'Transaction Fee Tax (Rp)']].drop_duplicates('Order #')

order_epayment = ordol_real.merge(epayment_short, how = 'left', on = ['Order #'])
order_epayment['Transaction Fee'] = order_epayment['Transaction Fee'].fillna(0) * order_epayment['Total Net Before PPN']/order_epayment.groupby(['Order #'])['Total Net Before PPN'].transform('sum')
order_epayment['Transaction Fee Tax (Rp)'] = order_epayment['Transaction Fee Tax (Rp)'].fillna(0) * order_epayment['Total Net Before PPN']/order_epayment.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

bundle_merge = ordol_bundle.merge(order_epayment[['Order #', 'Product Name','Transaction Fee', 'Transaction Fee Tax (Rp)']].drop_duplicates(['Order #', 'Product Name']), how = 'left', 
                                 left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])
bundle_merge = bundle_merge.rename(columns = {'Product Name_x' : 'Product Name'})
bundle_merge = bundle_merge.drop('Product Name_y', axis = 1)

bundle_merge['Transaction Fee'] = bundle_merge['Transaction Fee'].fillna(0) * bundle_merge['Total Net Before PPN']/bundle_merge.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundle_merge['Transaction Fee Tax (Rp)'] = bundle_merge['Transaction Fee Tax (Rp)'].fillna(0) * bundle_merge['Total Net Before PPN']/bundle_merge.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')

order_epayment = order_epayment.append(bundle_merge, ignore_index = True, sort = False)

order_epayment['Payment Fee'] = order_epayment['Transaction Fee'] + order_epayment['Transaction Fee Tax (Rp)']

order_epayment['Shipping Fee'] = order_epayment['Discount Voucher Shipping']

order_aft_epayment = order_epayment.append(non_epayment, ignore_index = True, sort = False)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [242]:
## run -24
order_aft_epayment['File Payment'] = np.nan

indeks = order_aft_epayment[order_aft_epayment['Order #'].isin(transfer['Order #'])].index.to_list()
order_aft_epayment['File Payment'][indeks] = 'Transfer'

indeks = order_aft_epayment[order_aft_epayment['Order #'].isin(cod['Order #'])].index.to_list()
order_aft_epayment['File Payment'][indeks] = 'COD'

indeks = order_aft_epayment[order_aft_epayment['Order #'].isin(epayment['Order #'])].index.to_list()
order_aft_epayment['File Payment'][indeks] = 'E-Payment'

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [243]:
## run -25
order_aft_epayment['Biaya Produk'] = order_aft_epayment['Real Price'] - order_aft_epayment['Discount Voucher Product']

In [244]:
### only display, tentative
pd.options.display.max_columns = 150

In [245]:
## run -26
order_aft_epayment['Paid'] = np.nan

orders = order_aft_epayment[(order_aft_epayment['Transfer'].notnull())]['Order #'].unique()

indeks = order_aft_epayment[order_aft_epayment['Order #'].isin(orders)].index.to_list()

order_aft_epayment['Paid'][indeks] = 'Paid'

cek_cod_payment = pd.read_excel(r"Settlement Project\Order Online\Settement transfer, Kode Voucher\Cek Settlement Homdel COD dan e-payment 2022 - ok.xlsx", sheet_name = cek_cod.title())
cek_cod_payment2 = pd.read_excel(r"Settlement Project\Order Online\Settement transfer, Kode Voucher\04. Settlement NHD 2023.xlsx", sheet_name = cek_cod.title())
cek_cod_payment=cek_cod_payment.append(cek_cod_payment2, ignore_index = True, sort = False)

cek_cod_payment['Order #'] = cek_cod_payment['Description'].astype(str).str.split('#', expand = True)[1].str.split(')', expand = True)[0].astype(str)
orders = order_aft_epayment[
    (order_aft_epayment['Order #'].astype(str).isin(cek_cod_payment['Order #'].unique()))
]['Order #'].unique()

indeks = order_aft_epayment[order_aft_epayment['Order #'].isin(orders)].index.to_list()

order_aft_epayment['Paid'][indeks] = 'Paid'
# print(order_aft_epayment.groupby(['Year','Month','Paid'])[['Order #']].nunique())
order_aft_epayment['Paid'] = order_aft_epayment['Paid'].fillna('Unpaid')
# print(order_aft_epayment.groupby(['Year','Month','Paid'])[['Order #']].nunique())

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [246]:
### check paid -> unpaid checkkk
cari="105225728"

# # transfer[transfer['Order #']=='40906863']
order_aft_epayment[(order_aft_epayment['Order #']==cari)&(order_aft_epayment['Bundle Name'].isnull())]#[['Real SKU','Real Nama Produk','Transfer','Real Price','Discount Voucher Product','Biaya Produk']]
# epayment[epayment['Order #']==cari]
# cek_cod_payment[cek_cod_payment['Order #']==cari]
# # transfer[transfer['Order #']=='40906863']
transfer[transfer['Order #']==cari]
# prev[prev['Order #']=='65934434']#.columns#['Paid March']
# ordol_merge[ordol_merge['Order #']==cari]
##
# order_aft_epayment[(order_aft_epayment['Year']==2022)&(order_aft_epayment['Month']=='August')].groupby(['File Payment','Paid'])[['Order #']].nunique()

,Tgl,Mutasi,Bank,617,Nama Konsumen,Order #,ID order Lama,Note,AWB,Tanggal Kirim,Tanggal,Area,Code Area,tgl,Tanggal Proses,Unnamed: 11,Note Utk Timo,Transfer


In [247]:
## run -27
tanpa_bayar1 = pd.read_excel(r'Settlement Project\Order Online/Settement transfer, Kode Voucher/Mutasi Rekening Order Online.xlsx', sheet_name = 'Rekap Tanpa Bayar')
tanpa_bayar2 = pd.read_excel(r'Settlement Project\Order Online/Settement transfer, Kode Voucher/Order Online-Transfer.xlsx', sheet_name = 'Rekap Tanpa Bayar NHD')
tanpa_bayar3 = pd.read_excel(r"Settlement Project\Order Online\Settement transfer, Kode Voucher\Mutasi Rekening (18).xlsx", sheet_name = 'Rekap Tanpa Bayar NHD')

tanpa_bayar2 = tanpa_bayar2[~tanpa_bayar2['Order #'].isin(tanpa_bayar1['Order #'])]
tanpa_bayar3 = tanpa_bayar3[~tanpa_bayar3['Order #'].isin(tanpa_bayar2['Order #'])]

tanpa_bayar = tanpa_bayar1.append(tanpa_bayar2, ignore_index = True, sort = False).append(tanpa_bayar3, ignore_index = True, sort = False)


tanpa_bayar = tanpa_bayar[tanpa_bayar['Channel'] == channel]
print(channel)
print(len(tanpa_bayar))

indeks = order_aft_epayment[order_aft_epayment['Order #'].isin(tanpa_bayar['Order #'])].index.to_list()
order_aft_epayment['Paid'][indeks] = 'Paid'

cancel1 = pd.read_excel(r"Settlement Project\Order Online\Settement transfer, Kode Voucher\Mutasi Rekening Order Online.xlsx", sheet_name = 'Transaksi Cancel')
cancel2 = pd.read_excel(r"Settlement Project\Order Online\Settement transfer, Kode Voucher\Order Online-Transfer.xlsx", sheet_name = 'Transaksi Cancel NHD')
cancel3 = pd.read_excel(r"Settlement Project\Order Online\Settement transfer, Kode Voucher\Mutasi Rekening (18).xlsx", sheet_name = 'Transaksi Cancel NHD')
if 'Order#' in cancel3.columns:
    cancel3=cancel3.rename(columns={'Order#':'Order #'})

cancel2 = cancel2[~cancel2['Order #'].isin(cancel1['Order #'])]
cancel3 = cancel3[~cancel3['Order #'].isin(cancel2['Order #'])]
cancel = cancel1.append(cancel2, ignore_index = True, sort = False).append(cancel3, ignore_index = True, sort = False)
cancel['Order #']=cancel['Order #'].astype(str)
cancel = cancel[cancel['Channel'] == channel]
print(channel)
print(len(cancel))

indeks = order_aft_epayment[order_aft_epayment['Order #'].isin(cancel['Order #'])].index.to_list()
order_aft_epayment['Paid'][indeks] = 'Cancel'

Order Online Samarinda
1


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Order Online Samarinda
48


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [248]:
# order_aft_epayment[order_aft_epayment['Order #']=='109120925']
# # cancel[cancel['Order #']==109120925]
# cancel['Order #'].unique()
# # order_aft_epayment['Order #'].unique()

In [249]:
### run -28

total_trx = order_aft_epayment.copy()

total_trx['Product ID'] = total_trx['Real SKU']
total_trx['SO STATUS'] = total_trx['Status Seller Centre']
total_trx['Warehouse Quantity'] = total_trx['Quantity']
total_trx['Fullfee Prop'] = np.nan
total_trx['Comfee Prop'] = np.nan
total_trx['WH'] = np.nan
total_trx['Warehouse Origin'] = np.nan

total_trx = total_trx.rename(columns = {
    'Real Price' : 'Total Selcen',
    'Status Seller Centre' : 'Order Status_y',
    'Biaya Produk' : 'Total Produk Prop',
    'created_at' : 'Order Date Seller Centre',
    'Discount Voucher Product' : 'Voucher Product',
    'Discount Voucher Shipping' :'Voucher Ongkir'
})

total_trx['Total Produk Prop Adj'] = total_trx['Total Produk Prop']/1.11
total_trx['BTL Cost'] = (total_trx['Total Produk Prop Adj']) - total_trx['Total Net Before PPN']
total_trx['Payment Fee'] = total_trx['Payment Fee'] * (-1)

total_trx_full = total_trx[['Order #',
 'Sales Order ID',
 'AWB',
 'Paid Date',
 'Order date',
 'Week',
 'Date',
 'Month',
 'Quarter',
 'Year',
 'Channel',
 'SKU',
 'Brand',
 'Product Name',
 'Bundle Name',
 'Price List NFI',
 'Qty. Invoiced',
 'Total Net',
 'Sub Brand',
 'Real SKU',
 'Real Nama Produk',
 'Parent Item',
 'Parent SKU',
 'Bundle Flag',
 'Shipping Courier',
 'Store',
 'True datetime',
 'Store Type',
 'Region Group',
 'PL Before PPN',
 'Total Net Before PPN',
 'Category Baru',
 'Exported Parent Item',
 'Order Date Seller Centre',
 'Product ID',
 'Total Selcen',
 'Order Status_y',
 'Warehouse Origin',
 'Paid',
 'SO STATUS',
 'WH',
 'Warehouse Quantity',
 'Total Produk Prop',
 'Fullfee Prop',
 'Comfee Prop',
 'BTL Cost',
 'Total Produk Prop Adj',
 'Payment Fee',
  'Shipping Fee',
  'Voucher Product',
  'Voucher Ongkir','Coupon Code',
  'Pemilik Budget',
  'Cluster'
 ]]

total_trx_full = total_trx_full.rename(columns = {
    'Total Selcen' : 'Total Seller Centre', 
    'Order Status_y' : 'Order Status Seller Centre',
    'SO STATUS' : 'Status Gudang'
})

In [250]:
## just checking

len(total_trx_full)
total_trx_full.groupby(['Year','Month','Paid'])[['Order #']].nunique()#[['Total Net Before PPN','Total Produk Prop']].sum()
### knp cman paid unpaid ya wkwkwk
# total_trx_full[total_trx_full['Paid']=='Unpaid']
# result[result['Year']==2022].groupby(['Year','Month','Paid'])[['Order #']].nunique()
# total_trx_full[(total_trx_full['Year']==2022)&(total_trx_full['Month']=='August')][['Order #','Order date','Total Net Before PPN','Total Produk Prop','Paid']]

Order #
Year   Month    Paid           
2023.0 April    Paid         35
                Unpaid        2
       February Paid         31
                Unpaid        4
       January  Paid         19
       March    Paid         28
                Unpaid       13
       May      Paid         11

In [251]:
####  run 29 -jangan lupa ganti nama file di sini

if '2020' in kota:
    total_trx_full.to_csv(r'Settlement Project/Output File/Order Online/2020/' + 'Running Order Online ' + kota + ' Mei Dec 2020.csv', index = False, sep = ';')
    print('2020')
else :
    total_trx_full.to_csv(r'Settlement Project\Backup\Realisasi May 2023\Final Result\Before Compare\Order Online per City/' + 'Running Order Online ' + kota + ' Jan 21 Apr 23.csv', index = False, sep = ';')
    print('2022')

2022


In [252]:
#### back to pilih kota yaaaa

In [253]:
total_trx_full#[total_trx_full['Order #']=='82323159']
# result[result['Order #']=='73856858']

,Order #,Sales Order ID,AWB,Paid Date,Order date,Week,Date,Month,Quarter,Year,Channel,SKU,Brand,Product Name,Bundle Name,Price List NFI,Qty. Invoiced,Total Net,Sub Brand,Real SKU,Real Nama Produk,Parent Item,Parent SKU,Bundle Flag,Shipping Courier,Store,True datetime,Store Type,Region Group,PL Before PPN,Total Net Before PPN,Category Baru,Exported Parent Item,Order Date Seller Centre,Product ID,Total Seller Centre,Order Status Seller Centre,Warehouse Origin,Paid,Status Gudang,WH,Warehouse Quantity,Total Produk Prop,Fullfee Prop,Comfee Prop,BTL Cost,Total Produk Prop Adj,Payment Fee,Shipping Fee,Voucher Product,Voucher Ongkir,Coupon Code,Pemilik Budget,Cluster
0,96342637,96342637,0,NaN,2023-04-01 09:51:00,1.0,4.0,January,1.0,2023.0,Order Online Samarinda,2101551180,HiLo,HiLo Gold Chocolate 500gr,NaN,77700.0,24.0,1864800.0,HILO GOLD,2101551180,HiLo Gold Chocolate 500gr,Hilo Gold Chocolate 500 Gr,NaN,NaN,JNE - CTC,Order Online,2023-01-04 09:51:00,Organic,Kalimantan Timur,70000.0,1680000.0,HILO GOLD,HILO GOLD CHOCOLATE 12DX500G,04-01-2023 - 09:51,2101551180,1.781265e+06,completed,NaN,Paid,completed,NaN,24.0,1.781265e+06,NaN,NaN,-7.525664e+04,1.604743e+06,-12469.321028,124034.278181,0.0,124034.278181,SMDRSLB016,Homdel,NaN
1,96342637,96342637,0,NaN,2023-04-01 09:51:00,1.0,4.0,January,1.0,2023.0,Order Online Samarinda,2101584180,HiLo,HiLo Gold Vanilla 500gr,NaN,77700.0,6.0,466200.0,HILO GOLD,2101584180,HiLo Gold Vanilla 500gr,Hilo Gold Vanilla 500G,NaN,NaN,JNE - CTC,Order Online,2023-01-04 09:51:00,Organic,Kalimantan Timur,70000.0,420000.0,HILO GOLD,HILO GOLD VANILLA 12DX500G,04-01-2023 - 09:51,2101584180,4.453163e+05,completed,NaN,Paid,completed,NaN,6.0,4.453163e+05,NaN,NaN,-1.881416e+04,4.011858e+05,-3117.330257,31008.569545,0.0,31008.569545,SMDRSLB016,Homdel,NaN
2,96342637,96342637,0,NaN,2023-04-01 09:51:00,1.0,4.0,January,1.0,2023.0,Order Online Samarinda,2102040225,TS,Tropicana Slim Gula Jawa 350ml,NaN,61050.0,6.0,366300.0,TS KUNING OTHERS,2102040225,Tropicana Slim Gula Jawa 350ml,Tropicana Slim Gula Jawa 350ml,2.10204e+09,NaN,JNE - CTC,Order Online,2023-01-04 09:51:00,Organic,Kalimantan Timur,55000.0,330000.0,TS KUNING OTHERS,TS GULA JAWA 12BX350ML,04-01-2023 - 09:51,2102040225,3.498914e+05,completed,NaN,Paid,completed,NaN,6.0,3.498914e+05,NaN,NaN,-1.478255e+04,3.152174e+05,-2449.330916,24363.876071,0.0,24363.876071,SMDRSLB016,Homdel,NaN
3,96342637,96342637,0,NaN,2023-04-01 09:51:00,1.0,4.0,January,1.0,2023.0,Order Online Samarinda,2101484302,HiLo,HiLo School Susu Vanilla (10sch) Gusset,NaN,44400.0,8.0,355200.0,HILO SCHOOL,2101484302,HiLo School Susu Vanilla (10 sch) Gusset,HiLo School Vanilla Gusset (10 Sch),NaN,NaN,JNE - CTC,Order Online,2023-01-04 09:51:00,Organic,Kalimantan Timur,40000.0,320000.0,HILO SCHOOL,HILO SCHOOL SUSU VANILLA 8GUSSETX10SX35G,04-01-2023 - 09:51,2101484302,3.392886e+05,completed,NaN,Paid,completed,NaN,8.0,3.392886e+05,NaN,NaN,-1.433460e+04,3.056654e+05,-2375.108767,23625.576796,0.0,23625.576796,SMDRSLB016,Homdel,NaN
4,96342637,96342637,0,NaN,2023-04-01 09:51:00,1.0,4.0,January,1.0,2023.0,Order Online Samarinda,2104589105,TS,Tropicana Slim Klepon Cookies (5 sch),NaN,22200.0,4.0,88800.0,TS KUNING OTHERS,2104589105,Tropicana Slim Klepon Cookies (5 Sch),Tropicana Slim Klepon Cookies (5 Sch),NaN,NaN,JNE - CTC,Order Online,2023-01-04 09:51:00,Organic,Kalimantan Timur,20000.0,80000.0,TS KUNING OTHERS,TS COOKIES KLEPON 12DX5SX20G,04-01-2023 - 09:51,2104589105,8.482215e+04,completed,NaN,Paid,completed,NaN,4.0,8.482215e+04,NaN,NaN,-3.583650e+03,7.641635e+04,-593.777192,5906.394199,0.0,5906.394199,SMDRSLB016,Homdel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,105729682,105729682,0,NaN,2023-03-31 09:47:00,13.0,31.0,March,1.0,2023.0,Order Online Samarinda,2104523105,TS,Tropicana Slim Hokkaido Cheese 100gr,NaN,23310.0,2.0,46620.0,TS KUN

# Joining

In [254]:
### run -30 ganti nama folder yaa
import pandas as pd
import os

result = pd.DataFrame()

for i in os.listdir('Settlement Project\Backup\Realisasi May 2023\Final Result\Before Compare\Order Online per City'):
    if 'Total' not in i and 'csv' in i:
        result = result.append(pd.read_csv("Settlement Project\Backup\Realisasi May 2023\Final Result\Before Compare\Order Online per City/" + i, sep = ';'), ignore_index = True, sort = False)
        print(i)
        # print('D:\Backup\Backup 26 Mei\Settlement Project Backup\Realisasi Maret 2022\Final Result\Order Online Per City/' + i)

Running Order Online Bali Jan 21 Apr 23.csv
Running Order Online Banjarmasin Jan 21 Apr 23.csv
Running Order Online Jawa Barat Jan 21 Apr 23.csv
Running Order Online Jawa Tengah Jan 21 Apr 23.csv
Running Order Online Jawa Timur Jan 21 Apr 23.csv
Running Order Online Kaltim Jan 21 Apr 23.csv
Running Order Online Lampung Jan 21 Apr 23.csv
Running Order Online Makasar Jan 21 Apr 23.csv
Running Order Online Medan Jan 21 Apr 23.csv
Running Order Online Palembang Jan 21 Apr 23.csv
Running Order Online Pekanbaru Jan 21 Apr 23.csv


In [255]:
## run -31 ganti file bulan lalu, ganti nama bulan di kolom paid xxxx
import pandas as pd

prev = pd.read_csv(r"D:\6. settlement\Code\Settlement Project\Backup\Realisasi Apr 2023\Final Result\Raw Unpivoted\CSV\Runningan Order Online Total Jan 21 - Apr 2023.csv", sep = ';')
current = result.copy()

prev['Order #'] = prev['Order #'].astype(str)
prev['Real SKU'] = prev['Real SKU'].astype(str)
current['Order #'] = current['Order #'].astype(str)
current['Real SKU'] = current['Real SKU'].astype(str)

merged = current.merge(prev[['Order #','Real SKU', 'Paid April', 'Bundle Name']].drop_duplicates(['Order #', 'Real SKU', 'Bundle Name']), how = 'left', on = ['Order #', 'Real SKU', 'Bundle Name'])
import numpy as np
# print(merged['Paid September'].unique())
merged['Paid Beda'] = np.nan

indeks = merged[merged['Paid'] != merged['Paid April']].index.to_list()
merged['Paid Beda'][indeks] = 'Beda'

merged['Paid Beda'] = merged['Paid Beda'].fillna('Sama')
# merged[merged['Paid Beda'] == 'Beda'][['Order #','Brand','Real SKU','Month', 'Paid_x', 'Paid_y', 'Paid Beda']]
merged[merged['Paid Beda'] == 'Beda'].groupby(['Year', 'Month', 'Paid', 'Paid April'], dropna = False)['Order #'].nunique()
# merged.groupby(['Year', 'Month', 'Paid', 'Paid March'], dropna = False)['Order #'].nunique()

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Year    Month     Paid    Paid April
2023.0  April     Cancel  Unpaid          1
                  Paid    Unpaid         50
                          NaN           138
                  Unpaid  NaN            18
        February  Cancel  Paid            4
                  Paid    Unpaid          1
        March     Paid    Unpaid          1
        May       Cancel  NaN             1
                  Paid    NaN           464
                  Unpaid  NaN            80
Name: Order #, dtype: int64

In [168]:
# prev[prev['Paid March'].isnull()]
# prev[prev['Order #']==102374044]
current[(current['Month']=='May')]['Date'].max()

23.0

In [169]:
# print('realisasi Agustus')
# display(prev[prev['Order #']=="69429141"][['Real SKU','SKU','Real Nama Produk','Bundle Name','Qty. Invoiced','Paid August','Paid July','Paid Beda']])
# print('realisasi September')
# merged[merged['Order #']=="69429141"][['Real SKU','SKU','Real Nama Produk','Bundle Name','Qty. Invoiced','Paid September','Paid August','Paid Beda']]

In [256]:
## run 32
from datetime import date
merged = merged.rename(columns = {'Paid' : 'Paid May'})


In [257]:
merged.loc[(merged['Year']==2023)&(merged['Month']=='May')&(merged['Date'].isin([22,23])),'Paid May']='Unpaid'

In [258]:
## run 33
finres=pd.read_excel("Settlement Project\Backup\Realisasi Feb 2023\Final Result\Ke Mbak May\Runningan Order Online Total Jan 21 - Feb 2023.xlsx",sheet_name='Running Order Online Total')

In [259]:
## run 34
finres=finres.rename(columns={'Paid January':'Paid April','Paid February':'Paid May',})
# finres=finres.rename(columns={'Paid March':'Paid August'})
merged[finres.columns].to_excel(f'Settlement Project\Backup\Realisasi May 2023\Final Result\Raw Unpivoted\Excel\Runningan Order Online Total Jan 2023 - May 2023 rev1.xlsx', index = False)
merged[finres.columns].to_csv(f'Settlement Project\Backup\Realisasi May 2023\Final Result\Raw Unpivoted\CSV\Runningan Order Online Total Jan 2023 - May 2023 rev1.csv',sep=';', index = False)

In [ ]:
#### selesai :) #####

# Check Status

In [ ]:
import pandas as pd

result = pd.read_csv(r"D:\settlement\Code\Settlement Project\Backup\Realisasi Mei 2022\Final Result\Before Compare\Order Online per City\Running Order Online Banjarmasin Jan 21 Mei 22.csv", sep = ';')

In [ ]:
result[result['Order Status Seller Centre'].isnull()].groupby(['Channel', 'Real Nama Produk', 'Bundle Name'])['Order #'].nunique().reset_index()

In [ ]:
result[result['Order #']==73855997]#.columns